<a href="https://colab.research.google.com/github/flaviovdf/deep-ufmg/blob/main/listas/03-Redes-Neurais-em-PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preâmbulo

Imports, funções, downloads e instalação do Pytorch.

In [ ]:
# Basic imports.
import os
import time
import numpy as np
import torch

from torch.autograd import Variable
import torch.nn.functional as F

from torch import nn
from torch import optim

from torch.utils.data import DataLoader
from torch.utils import data
from torch.backends import cudnn

from torchvision import models
from torchvision import datasets
from torchvision import transforms

from skimage import io

from sklearn import metrics

from matplotlib import pyplot as plt

%matplotlib inline

cudnn.benchmark = True

## Casting para o dispositivo correto

Como usaremos processamento vetorial principalmente em GPUs para aprendizado profundo, primeiramente é possível verificar se há uma GPU disponível com o trecho de código abaixo, armazenando os tensores nos dispositivos apropriados.

In [ ]:
# Checking if GPU/CUDA is available.
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(device)

# Intro MLP

## Neurônios e a camada `nn.Linear`

A camada Linear do Pytorch ([nn.Linear](https://pytorch.org/docs/stable/nn.html#torch.nn.Linear)) é responsável por aplicar uma transformação linear no dado de entrada. Esta camada recebe como parâmetro a dimensão (número de *features*) da entrada e da saída (que na verdade, representa o número de neurônios dessa camada). Por padrão o bias já é incluído. **Um** perceptron pode ser facilmente representado como a seguir, desconsiderando a função de ativação:

```
linear = nn.Linear(in_dimension, 1)
```
Mas de uma forma geral, uma camada Linear com diversas *features* de entrada e diversas *features* de saída pode ser representada como:
```
nn.Linear(in_features, out_features)
```
![](https://github.com/flaviovdf/deep-ufmg/blob/main/listas/figs/nn_linear.png?raw=1)

In [ ]:
linear = nn.Linear(2, 1)
print(linear)

Como é possível ver no código abaixo, o Pytorch já inicia os pesos da camada aleatoriamente.

In [ ]:
for param in linear.parameters():
  print(param)

O **forward** consiste em passar seu dado de entrada pela rede, gerando um resultado ao final. Considerando a camada linear instanciada anteriormente, o resultado do forward é o mesmo do somatório da multiplicação de seus pesos pelas respectivas entradas juntamente com o bias, ou seja:

$$f_w(x) = w_0 + w_1x_1 + w_2x_2 + \ldots + w_nx_n$$

No Pytorch, realizamos o **forward** chamando o objeto onde nossa rede/modelo está instanciada, conforme exemplo abaixo.

In [ ]:
linear = nn.Linear(2, 1)
X = torch.tensor([2.0, 3.0]) # dado de entrada de exemplo considerando o perceptron definido como nn.Linear(2,1)
print('Pytorch: ', linear(X))

# acessamos os pesos do modelo com .weight e o bias com .bias
print('Manual: ', torch.sum(X * linear.weight) + linear.bias)

In [ ]:
# Ao fazermos linear(X), estamos fazendo implicitamente uma chamada na função forward da Linear
print('Forward com chamada implícita:', linear(X))
print('Forward com chamada explícita:', linear.forward(X))

## Exemplo de uma rede neural simples (1 camada)

O código abaixo cria uma rede neural simples usando `nn.Linear` e implementa o fluxo de treinamento para essa rede, ou seja, faz o forward, calcula a loss, e otimiza seus pesos. Invista um pouco de tempo para entender a célula abaixo pois usaremos essa ideia para implementar a função de treino mais a frente.

In [ ]:
# Iremos criar dados que seguem a função y = 2x + 3
dataset = []
for x in range(10):
    dataset.append((x, 2*x + 3)) # tupla com (x, y)

In [ ]:
dataset

In [ ]:
# Definiremos uma loss (erro quadrático -> (y - y_hat)^2)
def loss_fn(predict, label):
    return torch.pow(label - predict, 2)

In [ ]:
linear = nn.Linear(1, 1) # Camada linear com 1 feature de entrada (mais o bias) e uma de saída
linear.to(device) # casting do linear para GPU

learning_rate = 0.01
print(f'Parâmetros iniciais: {list(linear.parameters())}\n')

for epoch in range(100):
    epoch_loss = 0

    for X, y in dataset:
        # Fazendo o casting dos dados para tensores na GPU
        X = torch.FloatTensor([X]).to(device)
        y = torch.FloatTensor([y]).to(device)

        y_pred = linear(X)  # etapa de forward
        loss = loss_fn(y_pred, y)  # calcula a loss
        epoch_loss += loss.item()

        # Etapa de backprop
        loss.backward()
        with torch.no_grad(): # não queremos fazer com que o pytorch anote as operações do backprop e acumule o gradiente
            for param in linear.parameters():
                param -= learning_rate * param.grad  # atualização dos parametros (pesos e bias) com base no gradiente
                param.grad.zero_()  # resetando o gradiente

    if (epoch + 1) % 10 == 0:
        print("Epoch {} - loss: {}".format(epoch + 1, epoch_loss))

print('\nParâmetros finais: ', list(linear.parameters()))

In [ ]:
X = torch.FloatTensor([20]).to(device)
print(linear(X)) # forward do valor 20 para conferir resultado, saida deve ser aproximadamente = 2x+3 = 2*20+3 = 43

## O módulo `nn.Sequential`

Na prática, criaremos redes com diversas camadas. O bloco `nn.Sequential` permite agrupar as camadas de forma sequencial para que o forward seja realizado na ordem desejada. Veja um exemplo para um *Multilayer Perceptron (MLP)* abaixo.

In [ ]:
in_features = 28
out_features = 8

first_hidden_size = 64
second_hidden_size = 32

MLP = nn.Sequential(
    nn.Linear(in_features, first_hidden_size), nn.ReLU(),
    nn.Linear(first_hidden_size, second_hidden_size), nn.ReLU(),
    nn.Linear(second_hidden_size, out_features)
)

print(MLP)

In [ ]:
test_data = torch.randn((10, 28)) # 10 dados de input aleatórios com 28 features

output = MLP(test_data) # forward da rede
print(output.size())

Informação sobre outras camadas lineares, como nn.Bilinear e nn.Identity, podem ser vistas na documentação: https://pytorch.org/docs/stable/nn.html#linear-layers

## Conjunto de Exercícios 1 - Implementação de uma MLP

Vamos agora treinar um MLP simples no dataset de [Breast Cancer da UCI](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)). A célula abaixo irá carregar os dados utilizando a biblioteca scikit-learn, que já possui o dataset pronto para utilizarmos.

- Neste exercício, não estamos interessados em trabalhar com dados de treino/teste, mas sim apenas estudar como definir uma rede neural de múltiplas camadas em PyTorch e realizar o treinamento dos seus pesos e viéses.

In [ ]:
from sklearn.datasets import load_breast_cancer

X, y = load_breast_cancer(return_X_y=True)

# Convertendo os dados para tensores PyTorch e fazendo o casting para o dispositivo apropriado
X = torch.FloatTensor(X).to(device)
y = torch.FloatTensor(y).to(device)

# Modificando o shape das anotações para ser um vetor (n, 1) para não gerar erros no cálculo da função de perda
# Isso é necessário pois os produtos internos feitos no PyTorch irá resultar em uma matriz (n, 1), ou seja, n-linhas
# onde cada linha terá um valor binário (classe daquela observação)
y = y.reshape(y.shape[0], 1)

# Exibindo o tamanho dos tensores
print('Tamanho de X: ', X.size())
print('Tamanho de y: ', y.size())

In [ ]:
print('Labels dos dados:', torch.unique(y))

1. Implemente na célula abaixo uma MLP, de nome **model**, utilizando o módulo `nn.Sequential`. A sua rede MLP deve possuir, pelo menos, uma camada escondida, usando uma ReLU como função de ativação entre as camadas.

    - Note que o nosso problema é um problema de classificação binária. Iremos utilizar uma função de perda do PyTorch que recebe como entrada a saída "crua" (*logits*) da sua rede neural. Sendo assim, não se preocupe em aplicar alguma transformação na saída (como uma sigmóide) para converter os valores para uma probabilidade, por exemplo.

In [ ]:
model = nn.Sequential(
    # implemente aqui a sua solução!
)

model.to(device)  # sempre é necessario fazer o casting da rede para jogá-la para GPU
print(model)

Abaixo, definimos uma loss e um otimizador usando o PyTorch. Não se preocupem como isso funciona agora, pois iremos ver em detalhes como definir e usar diferentes losses e otimizadores com o PyTorch.

In [ ]:
# Iremos utilizar a entropia cruzada binária como função de perda para o nosso problema
# Essa versão da BCE aceita como entrada a saída "crua" (logits) da sua rede neural.
# Outras versões, como a BCELoss aceita como entrada uma saída probabilística da sua rede neural (sigmoid ou softmax),
# sendo assim você deve colocar uma função de ativação depois do último nn.Linear do seu modelo.
# A versão BCEWithLogitsLoss implementa uma versão mais numericamente estável da loss, podemos observar isso na própria
# documentação do PyTorch: https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html?highlight=bcewithlogitsloss#torch.nn.BCEWithLogitsLoss
criterion = nn.BCEWithLogitsLoss()

In [ ]:
# Iremos utilizar o pacote optim para definir um otimizador que irá atualizar os pesos do modelo para nós.
# Aqui, utilizaremos SGD - Gradiente Descendente Estocástico.
# O pacote optim contém muitos outros algoritmos de otimização, porém, em todos o primeiro parâmetro irá dizer para os
# otimizadores quais tensores (com requires_grad=True) do nosso modelo ele deverá otimizar.
import torch.optim as optim

learning_rate = 1e-4
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

Abaixo teremos um loop de treinamento típico de PyTorch. Não precisa modificar em nada essa funçao, porém estude ela mesmo assim, já que usaremos esse fluxo como template para treinamento de outros modelos mais a frente no curso.

In [ ]:
# Aqui iremos criar uma lista de loss para cada época
loss_list = []

# Iterando sobre as épocas
n_epochs = 500
for epoch in range(n_epochs):
    preds = model(X)
    loss = criterion(preds, y)

    # Salvando a loss da iteração atual (para plots futuros)
    loss_list.append(loss.item())

    # Antes de fazermos o backward pass, iremos zerar o gradiente de todos os tensores
    # atrelados ao otimizador utilizando a chamada de função .zero_grad() do nosso otimizador.
    # Faremos isso pois os gradientes são acumulados, sempre que chamamos .backward(), em buffers nos
    # tensores que representam os pesos dos nossos modelos, ou seja, não são sobrescritos.
    # Para mais detalhes, você pode dar uma olhada na documentação do torch.autograd.backward
    optimizer.zero_grad()

    # Realizando o backward pass, ou seja, computando os gradientes da nossa função de perda
    # com respeito aos parâmetros (pesos) do nosso modelo
    loss.backward()

    # Chamando a função .step() do nosso otimizador para realizar um "passo" na otimização.
    # Nesse caso, o "passo" será realizar o cálculo que vimos do gradiente descendente
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch + 1}: loss = {loss.item():.5f}')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 8))

ax.set_title('Evolução da função de perda ao longo das épocas')
ax.set_ylabel('Valor da função de perda')
ax.set_xlabel('Épocas')
ax.plot(np.asarray(loss_list))

plt.show()